In [ ]:
import shutil
import os
import pandas as pd

# 파일 경로 (Windows 경로를 올바르게 처리)
folder_path = r"C:\VISSIM_Workspace\network_new\network_version2\network_version2.3_100"

interval_value = 60
from_time = 1800
# 시뮬레이션 횟수
siml_count = 10
mer_count = 0
inpx_name = "network01_mer"

destination_dir = os.path.join(folder_path, inpx_name) # 파일명_mer 수정해야함
os.makedirs(destination_dir, exist_ok=True)  # 이미 존재하면 생성하지 않음
# .mer 파일 복사
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]
print("총 .mer 파일 수 : ", len(mer_list))

for start in range(0, len(mer_list), siml_count):
    batch_files = mer_list[start:start+siml_count]
    df_list = []
    for file in batch_files:
        mer_path = os.path.join(destination_dir, file)
        print("mer_path : ", mer_path)
        # 파일 읽기
        with open(mer_path, "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
        # 데이터가 시작하는 인덱스 찾기
        data_start_idx = None

        for j, line in enumerate(lines):
            if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                data_start_idx = j
                break

        # 데이터프레임 생성
        if data_start_idx is not None:
            # 컬럼명 추출
            columns = lines[data_start_idx].strip().split(";")

            # 데이터 부분 추출 및 가공
            data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
            data = [line.strip().split(";") for line in data_lines if line.strip()]

            # 데이터프레임 생성
            df = pd.DataFrame(data, columns=columns)
            # 컬럼명을 정리 (공백 제거)
            df.columns = df.columns.str.strip()

            # 변환할 숫자형 컬럼 리스트
            numeric_columns = ["Measurem.", "t(Entry)", "t(Exit)", "VehNo", "Vehicle type",
                               "Line", "v[km/h]", "b[m/s2]", "Occ", "Pers", "tQueue", "VehLength[m]"]

            # 각 컬럼을 숫자로 변환
            for col in numeric_columns:
                if col in df.columns:  # 컬럼이 존재하는지 확인
                    df[col] = pd.to_numeric(df[col], errors="coerce")  # 변환 실패 시 NaN 처리
            df["time"] = df[["t(Entry)", "t(Exit)"]].max(axis=1)
            # 시간대별 범위 적용
            current_time = from_time
            max_time = df["time"].max()

            while current_time <= max_time:
                next_time = current_time + interval_value
                df.loc[df["time"].between(current_time, next_time, inclusive="right"), "time"] = next_time
                df.loc[df["time"] == 1800, "time"] = 1860
                current_time = next_time

            df = df.groupby(["Vehicle type", "Measurem.", "time"])[["b[m/s2]", "Occ", "tQueue", "VehLength[m]", "v[km/h]"]].agg("mean").reset_index().round(2)
            df_list.append(df)
    merged_df = pd.concat(df_list, ignore_index=True)
    # 같은 Vehicle type, Measurem., time  기준으로 평균
    avg_df = merged_df.groupby(["Vehicle type", "Measurem.", "time"])[["b[m/s2]", "Occ", "tQueue", "VehLength[m]", "v[km/h]"]].mean().reset_index().round(2)

    merout_path =os.path.join(destination_dir,"vehicle_output")
    os.makedirs(merout_path, exist_ok=True)  # 이미 존재하면 생성하지 않음

    last_file = xlsx_files = sorted([f for f in os.listdir(merout_path) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)

    if not xlsx_files:
        last_file = 1
    else:
        last_file = int(xlsx_files[-1].split("_")[0])+1
    excel_path = os.path.join(merout_path,f"{last_file}_vehicle_output.xlsx")
    avg_df.to_excel(excel_path, index=False)
    print("생성파일 : ", excel_path)
    mer_count += 1

    if mer_count >= len(mer_list)/siml_count:
        print("시뮬레이션 분석이 완료되었습니다.")
        break

총 .mer 파일 수 :  360
mer_path :  C:\VISSIM_Workspace\network_new\network_version2\network_version2.3_100\network01_mer\network02_수정_001.mer
